Reference paper: stability is stable 
Experiment: vary the value of rho and see the minimum sample size required for replicability

func getConvergenceSampleNum(hyperparams)
    For sample_num range(min_subsets_size, max_subsets_size, step):
        For range(repeat_num):
            random draw a subset of the dataset, whose size = sample_num
            Use algorithm 10 to get a model
        Check whether the models we get are 'replicable' to each other, accoding to the hyperparams. (see definition in page 14 of the paper)
        if converged, return current sample_num

func experiment(): 
# vary the rho and see minumum sample size required for replicability. 
# To make it simple, we can fix the value of other hyperparamers, such as alpha and beta.
    for rho in range(min_rho, max_rho, step):
        sample_num = getConvergenceSampleNum(rho)
        theoretical_sample_num = getTheoreticalSampleNum(rho) # compute according to algorithm 10 in the paper
        print("Hyperparams: ", hyperparams, "Sample size: ", sample_num)
    plot(hyperparams, theoretical_sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size
    plot(hyperparams, sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size

In [1]:
import pandas as pd
import numpy as np
import config
import Algorithm10 as a10
from sklearn.tree import export_text

##### Config Variables #####
np = <module 'numpy' from '/opt/anaconda3/envs/myenv/lib/python3.13/site-packages/numpy/__init__.py'>
dataset_path = ./dataset/Invistico_Airline.csv
model_path = ./models/
max_depth = 3
random_seed = 42
selected_features = ['Class', 'Seat comfort', 'Food and drink', 'Cleanliness', 'satisfaction']
rho = 0.1
alpha = 0.1
beta = 0.1
num_H = 10
get_m_up_bound = <function get_m_up_bound at 0x126b00360>
tau_up_bound = 0.0043429448190325185
tau = 0.0004342944819032519
m = 100
############################


In [2]:
data = pd.read_csv('dataset/Invistico_Airline.csv')
data.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0


In [3]:
data.shape

(129880, 22)

In [4]:
def are_trees_equal(tree1, tree2):
    # Check that both trees are fitted
    if not hasattr(tree1, 'tree_') or not hasattr(tree2, 'tree_'):
        raise ValueError("Both trees must be fitted before comparison.")

    # Compare parameters
    if tree1.get_params() != tree2.get_params():
        return False

    t1 = tree1.tree_
    t2 = tree2.tree_

    # Compare structure and splitting rules
    attributes_to_check = [
        'children_left', 'children_right',
        'feature', 'threshold',
        'impurity', 'n_node_samples', 'weighted_n_node_samples',
        'value'
    ]

    for attr in attributes_to_check:
        if not np.array_equal(getattr(t1, attr), getattr(t2, attr)):
            return False

    return True

In [5]:
X, y = a10.load_full_dataset(config.dataset_path, random_state=config.random_seed)
H = a10.build_candidate_trees(X, y,sample_size = 100000, max_depth=config.max_depth, num_trees=config.num_H, random_state=config.random_seed)    

for sample_size in range(10000, 20000 + 1, 500):
    qualifying_trees_list = []
    for i in range(2):
        res_trees = a10.replicable_learner(X, y, H, sample_size=sample_size, random_seed=config.random_seed+i)
        qualifying_trees_list.append(res_trees)
    #compare the percentage of tree that are equal in the two lists
    equal_trees = 0
    print("\nSample size: ", sample_size)
    print("Number of trees in the first list: ", len(qualifying_trees_list[0]))
    print("Number of trees in the second list: ", len(qualifying_trees_list[1]))
    smaller_list = min(len(qualifying_trees_list[0]), len(qualifying_trees_list[1]))
    for i in range(len(qualifying_trees_list[0])):
        for j in range(len(qualifying_trees_list[1])):
            if are_trees_equal(qualifying_trees_list[0][i], qualifying_trees_list[1][j]):
                equal_trees += 1
                continue
    print("Number of equal trees: ", equal_trees)

100%|██████████| 10/10 [00:00<00:00, 42.92it/s]


OPT error 0.24080000000000001 errors dict_values([0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001, 0.24080000000000001])
k: 56 v max candidates: 0.2649846740990639 v min candidates: 0.24109847759438502
[np.float64(0.24109847759438502), np.float64(0.2415327720762883), np.float64(0.24196706655819153), np.float64(0.24240136104009477), np.float64(0.24283565552199804), np.float64(0.24326995000390128), np.float64(0.24370424448580452), np.float64(0.2441385389677078), np.float64(0.24457283344961103), np.float64(0.2450071279315143), np.float64(0.24544142241341754), np.float64(0.24587571689532078), np.float64(0.24631001137722405), np.float64(0.2467443058591273), np.float64(0.24717860034103056), np.float64(0.2476128948229338), np.float64(0.24804718930483705), np.float64(0.24848148378674031), np.float64(0.24891577826864356), np.float64(0.2493500727505468), np.

In [6]:
def getConvergenceSampleNum(min_subset_size, max_subset_size, repeat_num, rho, sample_size_step=1):
    sample_size_replicablity_dict = {}
    
    X, y= a10.load_full_dataset(config.dataset_path, random_state=config.random_seed)
    for sample_size in range(min_subset_size, max_subset_size + 1, sample_size_step):
        #get dataset of size sample_size by sampling from the original dataset
        replicable_tree_list = []
        H = a10.build_candidate_trees(X, y,sample_size, max_depth=config.max_depth, num_trees=config.num_H, random_state=config.random_seed)    
        for i in range(repeat_num):
            print(f"sample size: {sample_size}, repeat: {i}")
            res_trees = a10.replicable_learner(X, y, H, sample_size, random_seed=config.random_seed+i)
            #print(f"number of res_trees: {len(res_trees)}")
            # check whether the candidate trees below v are same to each other
            #for a in range(len(res_trees)):
            #    for b in range(a + 1, len(res_trees)):
            #        if are_trees_equal(res_trees[a], res_trees[b]):
            #            print(f"tree {a} and tree {b} are the same")
            #        else:
            #            print(f"tree {a} and tree {b} are different")
            #            print("tree a",export_text(res_trees[a]),"tree b",export_text(res_trees[b]))
            tree = res_trees[0]
            replicable_tree_list.append(tree)
            
        #check the probability if the trees in the replicable_tree_list are the same
        same_tree_count = 0
        for i in range(len(replicable_tree_list)):
            for j in range(i + 1, len(replicable_tree_list)):
                
                if are_trees_equal(replicable_tree_list[i], replicable_tree_list[j]):
                    # print(f"tree {i} and tree {j} are the same")
                    same_tree_count += 1
        prob = same_tree_count / (repeat_num * (repeat_num - 1) / 2)
        sample_size_replicablity_dict[sample_size] = prob
        # if prob >= 1-rho:
        #     print(f"replicable at sample size: {sample_size}, prob: {prob}")
        #     return sample_size 
    return sample_size_replicablity_dict
        
    # # return sample_size_replicablity_dict
    # print(f"not replicable at sample size between {min_subset_size} and {max_subset_size}, prob: {prob}")
    # return -1

            
        
    

In [7]:
print("theoretical sample size: ", config.get_m_up_bound(config.num_H, config.rho, config.alpha, config.beta))
ans_dict = getConvergenceSampleNum(min_subset_size=100, max_subset_size=2000, repeat_num=10, rho=config.rho, sample_size_step=100)
for key, value in ans_dict.items():
    print(f"sample size: {key}, prob: {value}")
    if value >= 1 - config.rho:
        print(f"replicable at sample size: {key}, prob: {value}")
        break

theoretical sample size:  12231097.404690797


100%|██████████| 10/10 [00:00<00:00, 167.73it/s]

sample size: 100, repeat: 0
OPT error 0.21999999999999997 errors dict_values([0.27, 0.31000000000000005, 0.28, 0.21999999999999997, 0.24, 0.35, 0.30000000000000004, 0.24, 0.32999999999999996, 0.29000000000000004])
k: 56 v max candidates: 0.24418467409906383 v min candidates: 0.22029847759438498
[np.float64(0.22029847759438498), np.float64(0.22073277207628825), np.float64(0.2211670665581915), np.float64(0.22160136104009473), np.float64(0.222035655521998), np.float64(0.22246995000390124), np.float64(0.22290424448580448), np.float64(0.22333853896770775), np.float64(0.223772833449611), np.float64(0.22420712793151426), np.float64(0.2246414224134175), np.float64(0.22507571689532074), np.float64(0.225510011377224), np.float64(0.22594430585912725), np.float64(0.22637860034103052), np.float64(0.22681289482293376), np.float64(0.227247189304837), np.float64(0.22768148378674027), np.float64(0.22811577826864352), np.float64(0.22855007275054676), np.float64(0.22898436723245003), np.float64(0.2294186

OPT error 0.18999999999999995 errors dict_values([0.27, 0.30000000000000004, 0.21999999999999997, 0.19999999999999996, 0.18999999999999995, 0.24, 0.24, 0.21999999999999997, 0.28, 0.25])
k: 56 v max candidates: 0.21427355077797652 v min candidates: 0.19038735427329767
[np.float64(0.19038735427329767), np.float64(0.19082164875520094), np.float64(0.19125594323710418), np.float64(0.19169023771900742), np.float64(0.1921245322009107), np.float64(0.19255882668281393), np.float64(0.19299312116471717), np.float64(0.19342741564662044), np.float64(0.19386171012852368), np.float64(0.19429600461042695), np.float64(0.1947302990923302), np.float64(0.19516459357423344), np.float64(0.1955988880561367), np.float64(0.19603318253803995), np.float64(0.19646747701994322), np.float64(0.19690177150184646), np.float64(0.1973360659837497), np.float64(0.19777036046565297), np.float64(0.1982046549475562), np.float64(0.19863894942945945), np.float64(0.19907324391136272), np.float64(0.19950753839326596), np.float64

100%|██████████| 10/10 [00:00<00:00, 149.28it/s]


...

sample size: 1800, repeat: 0
OPT error 0.21166666666666667 errors dict_values([0.2316666666666667, 0.21166666666666667, 0.2316666666666667, 0.2283333333333334, 0.2316666666666667, 0.21166666666666667, 0.2283333333333334, 0.21166666666666667, 0.2316666666666667, 0.23333333333333328])
k: 56 v max candidates: 0.23585134076573053 v min candidates: 0.21196514426105167
[np.float64(0.21196514426105167), np.float64(0.21239943874295494), np.float64(0.21283373322485818), np.float64(0.21326802770676143), np.float64(0.2137023221886647), np.float64(0.21413661667056794), np.float64(0.21457091115247118), np.float64(0.21500520563437445), np.float64(0.2154395001162777), np.float64(0.21587379459818096), np.float64(0.2163080890800842), np.float64(0.21674238356198744), np.float64(0.2171766780438907), np.float64(0.21761097252579395), np.float64(0.21804526700769722), np.float64(0.21847956148960046), np.float64(0.2189138559715037), np.float64(0.21934815045340697), np.float64(0.2197824449353102), np.float64(0

100%|██████████| 10/10 [00:00<00:00, 147.24it/s]


sample size: 1900, repeat: 0
OPT error 0.2136842105263158 errors dict_values([0.23368421052631583, 0.2136842105263158, 0.22999999999999998, 0.22999999999999998, 0.23368421052631583, 0.2136842105263158, 0.22999999999999998, 0.2136842105263158, 0.2136842105263158, 0.2315789473684211])
k: 56 v max candidates: 0.23786888462537967 v min candidates: 0.21398268812070081
[np.float64(0.21398268812070081), np.float64(0.21441698260260408), np.float64(0.21485127708450732), np.float64(0.21528557156641057), np.float64(0.21571986604831384), np.float64(0.21615416053021708), np.float64(0.21658845501212032), np.float64(0.2170227494940236), np.float64(0.21745704397592683), np.float64(0.2178913384578301), np.float64(0.21832563293973334), np.float64(0.21875992742163658), np.float64(0.21919422190353985), np.float64(0.2196285163854431), np.float64(0.22006281086734636), np.float64(0.2204971053492496), np.float64(0.22093139983115284), np.float64(0.2213656943130561), np.float64(0.22179998879495935), np.float64(

100%|██████████| 10/10 [00:00<00:00, 155.22it/s]


sample size: 2000, repeat: 0
OPT error 0.21799999999999997 errors dict_values([0.238, 0.238, 0.238, 0.248, 0.238, 0.21799999999999997, 0.23250000000000004, 0.21799999999999997, 0.21799999999999997, 0.23950000000000005])
k: 56 v max candidates: 0.24218467409906383 v min candidates: 0.21829847759438498
[np.float64(0.21829847759438498), np.float64(0.21873277207628825), np.float64(0.2191670665581915), np.float64(0.21960136104009473), np.float64(0.220035655521998), np.float64(0.22046995000390124), np.float64(0.22090424448580448), np.float64(0.22133853896770775), np.float64(0.221772833449611), np.float64(0.22220712793151426), np.float64(0.2226414224134175), np.float64(0.22307571689532074), np.float64(0.223510011377224), np.float64(0.22394430585912725), np.float64(0.22437860034103052), np.float64(0.22481289482293376), np.float64(0.225247189304837), np.float64(0.22568148378674027), np.float64(0.2261157782686435), np.float64(0.22655007275054675), np.float64(0.22698436723245002), np.float64(0.22

In [8]:
print(ans_dict)

{100: 0.1111111111111111, 200: 0.8, 300: 1.0, 400: 1.0, 500: 1.0, 600: 1.0, 700: 1.0, 800: 1.0, 900: 1.0, 1000: 1.0, 1100: 1.0, 1200: 1.0, 1300: 1.0, 1400: 1.0, 1500: 1.0, 1600: 1.0, 1700: 1.0, 1800: 0.8, 1900: 1.0, 2000: 1.0}


In [14]:
print(config.rho)
config.rho = 0.5
print(np.log(np.log(100)))
print(np.exp(np.exp(3)),np.exp(3))

0.5
1.5271796258079011
528491311.4854943 20.085536923187668
